In [ ]:
%pip install nltk wordcloud xgboost lightgbm catboost 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [41]:
%pip uninstall leia-br -y
%pip install leia-br

Found existing installation: leia-br 0.0.1
Uninstalling leia-br-0.0.1:
  Successfully uninstalled leia-br-0.0.1
Note: you may need to restart the kernel to use updated packages.
  Using cached leia_br-0.0.1-py2.py3-none-any.whl.metadata (2.9 kB)
Using cached leia_br-0.0.1-py2.py3-none-any.whl (130 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [42]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# Text Processing Libraries
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from collections import Counter
from nltk import ngrams

# Sentiment Analysis
from LeIA import SentimentIntensityAnalyzer

# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Machine Learning Libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
)
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Evaluation Metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, classification_report, 
    confusion_matrix, roc_curve, roc_auc_score, ConfusionMatrixDisplay, auc
)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mumu1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mumu1\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [3]:
from IPython.core.display import HTML
HTML('''<style>
div.output_area {
    max-height: none !important;
}
</style>''')

# 출력 제한 완전히 제거
get_ipython().run_line_magic('config', "InlineBackend.print_figure_kwargs = {'bbox_inches': None}") 

In [4]:
from RFM import df_order_reviews, df_product_category_name_translation, df_products, merge_coi

# Merging

In [15]:
# 1) 전체 분석 review_comment_message, product_category_name, order_id
# 2) 이탈고객 vs 잠재우수 고객

df_reviews = df_order_reviews[['order_id', 'review_comment_message']]

# merge_coi + df_products = NLP
NLP = merge_coi.merge(
    df_products[['product_id', 'product_category_name']],  
    on='product_id',
    how='inner'
)

# NLP + df_reviews = NLP
NLP = NLP.merge(
    df_reviews[['order_id', 'review_comment_message']], 
    on='order_id',
    how='inner'
)


# Data cleaning

In [23]:
clean_NLP = NLP[['order_id', 'review_comment_message', 'product_category_name']]
# clean_NLP.isna().sum() # 0, 92940, 1522
# 결측치 제거
clean_NLP = clean_NLP.dropna(subset=['product_category_name', 'review_comment_message'])
# 중복 제거
clean_NLP = clean_NLP.drop_duplicates(subset=['review_comment_message']).reset_index(drop=True)
clean_NLP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33495 entries, 0 to 33494
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   order_id                33495 non-null  object
 1   review_comment_message  33495 non-null  object
 2   product_category_name   33495 non-null  object
dtypes: object(3)
memory usage: 785.2+ KB


# Text Preprocessing

In [ ]:
# Define Portuguese stopwords
STOP_WORDS = set(stopwords.words('portuguese'))


NameError: name 'stopwords' is not defined